In [ ]:
storageAccountName = ""
storageAccountAccessKey = ""
sasToken = ""

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print("OK!")
    except Exception as e:
      print("Falha", e)

In [ ]:
display(dbutils.fs.mounts())

In [ ]:
dbutils.fs.unmount('/mnt/datalake37e6a883b8e661af/lading-zone')
dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/bronze')
dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/silver')
dbutils.fs.unmount('/mnt/datalaked10514ad1cefdbcd/gold')

In [ ]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/landing-zone"))


Gerando um dataframe para cada arquivo a partir dos arquivos CSV gravado no container landing-zone do Azure Data Lake Storage


In [ ]:
df_apolice   = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/apolice.csv")
df_carro = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/carro.csv")
df_cliente   = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/cliente.csv")
df_endereco = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/endereco.csv")
df_estado    = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/estado.csv")
df_marca     = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/marca.csv")
df_modelo    = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/modelo.csv")
df_municipio = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/municipio.csv")
df_regiao    = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/regiao.csv")
df_sinistro = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/sinistro.csv")
df_telefone = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/telefone.csv")


Adicionando metadados de data e hora de processamento e nome do arquivo de origem

In [ ]:
from pyspark.sql.functions import current_timestamp, lit

df_apolice   = df_apolice.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("apolice.csv"))
df_carro     = df_carro.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("carro.csv"))
df_cliente   = df_cliente.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("cliente.csv"))
df_endereco  = df_endereco.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("endereco.csv"))
df_estado    = df_estado.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("estado.csv"))
df_marca     = df_marca.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("marca.csv"))
df_modelo    = df_modelo.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("modelo.csv"))
df_municipio = df_municipio.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("municipio.csv"))
df_regiao    = df_regiao.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("regiao.csv"))
df_sinistro  = df_sinistro.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("sinistro.csv"))
df_telefone  = df_telefone.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("telefone.csv"))


Salvando os dataframes em delta lake (formato de arquivo) no data lake (repositorio cloud)

In [ ]:
df_apolice.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/apolice")
df_carro.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/carro")
df_cliente.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/cliente")
df_endereco.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/endereco")
df_estado.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/estado")
df_marca.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/marca")
df_modelo.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/modelo")
df_municipio.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/municipio")
df_regiao.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/regiao")
df_sinistro.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/sinistro")
df_telefone.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/telefone")

In [ ]:
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze/"))

In [ ]:
spark.read.format('delta').load(f'/mnt/{storageAccountName}/bronze/endereco').limit(10).display()